# Recommendation Engine

## Importing libraries

In [16]:
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [17]:
### Getting Path of current file
import os
import sys
from pathlib import Path

path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/swethagayatrikandikonda/Downloads/Projects/SET PROJECT/Amazon-Product-Recommendation-System


## Reading the data

In [18]:
# Parse the current folder
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

# Reading the gzip file
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

def get_secondary_cat(arr):
  #text = eval(text)
  if(len(arr) > 1):
    return arr[1]
  else:
    return np.nan

In [19]:
from bs4 import BeautifulSoup

products_df = getDF(f'{path}/data/meta_Video_Games.json.gz')
products_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84814,"[Video Games, PlayStation Vita, Digital Games ...",,[<div>The Force is strong with this one The No...,,Lego Star Wars: The Force Awakens - PS Vita [D...,[],,by\n \n Warner Bros.,[],"[>#74,224 in Video Games (See Top 100 in Video...",[],Video Games,,,,B01HJ1521Y,[],[],{}
84815,"[Video Games, PlayStation 4, Digital Games & D...",,[<div>The Season Pass includes three Level Pac...,,Lego Star Wars: The Force Awakens Season Pass...,[],,by\n \n Warner Bros.,[DLC Requires base game],"[>#62,150 in Video Games (See Top 100 in Video...",[],Video Games,,,\n\t\t ...,B01HJ14TTA,[],[],{}
84816,"[Video Games, PlayStation 4, Digital Games & D...",,"[<div>The Technomancer takes you to Mars, wher...",,The Technomancer - PS4 [Digital Code],[],,by\n \n Focus Home Interactive,[],"[>#94,234 in Video Games (See Top 100 in Video...",[],Video Games,,,,B01HJ14OT0,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
84817,"[Video Games, Xbox 360, Accessories, </span></...",,[<b>FUNCTIONS:</b><br> 1.Take apart your Xbox ...,,"Repair T8 T6 Tools for XBOX One Xbox 360, YTTL...","[B01KBNB7K2, B06X6JSYPC, B01N6Y0Z7W, B06VXD2W5...",,by\n \n YTTL,[If you want to Replacement you Xbox one /360 ...,"[>#16,087 in Video Games (See Top 100 in Video...","[B01KH25ZY6, B00PG8SU26, B07G122BVS, B016XLTQP...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",B01HJC33WS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}


In [20]:
reviews_df = getDF(f'{path}/data/Video_Games.json.gz')
reviews_df = reviews_df.rename({'overall':'rating'},axis = 1).copy()
reviews_df = reviews_df[reviews_df['asin'].isin(products_df['asin'])]
# Drop any duplicates records based on reviewerID, asin and rating
reviews_df = reviews_df.drop_duplicates(subset = ['reviewerID','asin','rating']).copy()
reviews_df = reviews_df.dropna(subset=['asin', 'rating'])
reviews_df

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4.0,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1.0,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4.0,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2565344,5.0,True,"08 1, 2018",ANGB54K3888S4,B01HJEBIAA,josh,"Love it, work good",Works good,1533081600,NaN,NaN,NaN
2565345,5.0,True,"07 17, 2018",A3TEVKR0ZVQB2T,B01HJEBIAA,Prime Member,I do a lot of copy/paste and other keyboard sh...,Great mouse for work and gaming,1531785600,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...
2565346,5.0,True,"07 6, 2018",ABE7YPWEHNVJZ,B01HJEBIAA,Amazon Customer,One year in and it's still working great! Hig...,Five Stars,1530835200,NaN,NaN,NaN
2565347,5.0,True,"06 12, 2018",A3ES9QBK3G192O,B01HJEBIAA,Lina Marmolejos,EXCELENTE,Five Stars,1528761600,NaN,NaN,NaN


In [21]:
print("Number of Records in the data:", reviews_df.shape)

Number of Records in the data: (2487442, 12)


In [22]:
products_df.shape

(84819, 19)

## Examining the data

In [23]:
# Null values in the data
reviews_df.isna().sum()

rating                  0
verified                0
reviewTime              0
reviewerID              0
asin                    0
reviewerName          249
reviewText           1695
summary               797
unixReviewTime          0
vote              2064829
style             1209420
image             2459398
dtype: int64

In [24]:
(reviews_df['asin'].value_counts() > 100).sum()

5031

In [25]:
reviews_df['reviewerID'].nunique()

1539732

In [26]:
(reviews_df['reviewerID'].value_counts() > 30).sum()

1337

In [27]:
products_df.isna().sum()

category             0
tech1                0
description          0
fit                  0
title                0
also_buy             0
tech2                0
brand                0
feature              0
rank                 0
also_view            0
main_cat             0
similar_item         0
date                 0
price                0
asin                 0
imageURL             0
imageURLHighRes      0
details            107
dtype: int64

In [28]:
# Step 2: Filter products with at least 50 reviews from those active reviewers
product_counts = reviews_df['asin'].value_counts()
popular_products = product_counts[product_counts >= 50].index
reviews_df = reviews_df[reviews_df['asin'].isin(popular_products)]


In [29]:
products_df_final = products_df[products_df['asin'].isin(popular_products)]
products_df_final = products_df_final[products_df_final['main_cat']=='Video Games']

def clean_html_list(cell):
    if isinstance(cell, list):
        return [BeautifulSoup(str(item), "html.parser").get_text() for item in cell]
    return cell

products_df_final['description'] = products_df_final['description'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
products_df_final['category'] = products_df_final['category'].apply(clean_html_list)
products_df_final = products_df_final[~products_df_final['category'].apply(lambda x: 'Accessories' in x or 'Consoles' in x)]
products_df_final['secondary_cat'] = products_df_final['category'].apply(lambda x: get_secondary_cat(x)).fillna("")
products_df_final['description'] = products_df_final['description'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
products_df_final = products_df_final[products_df_final['title'].str.len() <= 30]


top_1000 = products_df_final['asin'].value_counts().nlargest(1000).index

# Step 2: Filter the DataFrame to include only those top 10 objects
filtered_products_df = products_df_final[products_df_final['asin'].isin(top_1000)].drop_duplicates(subset=['asin'])
filtered_products_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details,secondary_cat
38,"[Video Games, PC, Games]",,ANNO 2070BRAND NEW - IN STOCKDVD Rom Software ...,,Anno 2070,[],,Ubisoft,[A new era: while adhering to the fundamentals...,"[>#30,230 in Video Games (See Top 100 in Video...","[B013F0IP1C, B00JDP1AWU, B00XR3YC2E, B001AWMCM...",Video Games,,,$7.95,0700026657,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,PC
306,"[Video Games, Retro Gaming & Microconsoles, Ni...",,front label very worn Slam the pedal to the me...,,Cruis'n USA,"[B00002STGZ, B00002STI2, B00000DMAO, B00000F1G...",,Nintendo,[Nintendo 64 Classic Fun Racing Game],"[>#11,106 in Video Games (See Top 100 in Video...","[B00002STGZ, B00002STI2, B00000DMAX, B00000F1G...",Video Games,,,$12.34,B00000ID60,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{},Retro Gaming & Microconsoles
309,"[Video Games, Retro Gaming & Microconsoles, Ni...",,Mario Party [Japanese Import] for Nintendo 64 ...,,Mario Party,"[B000046Q7Q, B00000DMAX, B00005AW1W, B00002DHE...",,Nintendo,[Mini Games],"[>#3,000 in Video Games (See Top 100 in Video ...","[B00005AW1W, B00000DMAX, B0795F9PFR, B000046Q7...",Video Games,,,$59.59,B00000IFRI,[],[],{},Retro Gaming & Microconsoles
341,"[Video Games, Retro Gaming & Microconsoles, Ga...",,"It's the game that started a revolution, but i...",,Pokemon - Red Version,"[B00000J97G, B000047GEI, B0000296ZM, B00004TEY...",,by\n \n Nintendo,[],"[>#2,473 in Video Games (See Top 100 in Video ...","[B000047GEI, B07DLDCVG1, B00000J97G, B00004TEY...",Video Games,,,\n\t\t ...,B00000IYEQ,[],[],{},Retro Gaming & Microconsoles
344,"[Video Games, Retro Gaming & Microconsoles, Pl...",,Sonic has returned with his long-time pals Knu...,,Sonic Heroes - PlayStation 2,"[B0009PKDOY, B000C4088I, B0002EJ8FC, B0017HM0J...",,by\n \n Sega,"[1-2 Player, Team Sonic, Team Dark, Team Rose ...","[>#12,761 in Video Games (See Top 100 in Video...","[B0002EJ8FC, B000C4088I, B0009PKDOY, B0017HM0J...",Video Games,,,\n\t\t ...,B00000IZS1,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{},Retro Gaming & Microconsoles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58302,"[Video Games, Xbox 360, Games, ]",,Following their involvement in (REDACTED TO PR...,,Saints Row IV,"[B00N23Z8P4, B000ZKDOV2, B00EAXYHMK, B0050SYIL...",,by\n \n Deep Silver,[The American (Fever) Dream - Play as the Pres...,"[>#5,410 in Video Games (See Top 100 in Video ...","[B00L9OPNW6, B000ZKDOV2, B0050SYILE, B009DZTXR...",Video Games,,,\n\t\t ...,B00BRQN2EM,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{},Xbox 360
58303,"[Video Games, Xbox 360, Games, ]",,"Following an intense tour of duty, Jake Conway...",,Ride to Hell Retribution,"[B003O6E7EM, B003VWGBD4, B0030GBU32, B004A88W8...",,by\n \n Deep Silver,"[High-Speed Motorcycles that do stunts, as wel...","[>#26,065 in Video Games (See Top 100 in Video...","[B003VWGBD4, B003O6E7EM, B0058VI1SO, B000TP2V3...",Video Games,,,\n\t\t ...,B00BRQP2C2,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{},Xbox 360
58306,"[Video Games, PlayStation 3, Games, ]",,Following their involvement in (REDACTED TO PR...,,Saints Row IV,"[B0050SXKU4, B00N23Z8VI, B004QEWVLC, B009DZTXQ...",,by\n \n Deep Silver,[The American (Fever) Dream - Play as the Pres...,"[>#9,173 in Video Games (See Top 100 in Video ...","[B00L9OPPMO, B00N23Z8VI, B009DZTXQU, B0050SXKU...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",B00BRQN2H4,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{},PlayStation 3
58331,"[Video Games, Kids & Family, PlayStation 3, Ga...",,Disc Only,,Dev

In [30]:
reviews_df = reviews_df[reviews_df['asin'].isin((filtered_products_df['asin'].value_counts()).index)]

In [31]:
top_40k = reviews_df['reviewerID'].value_counts().nlargest(40000).index

# Step 2: Filter the DataFrame to include only those top 10 objects
filtered_reviews_df = reviews_df[reviews_df['reviewerID'].isin(top_40k)]
filtered_reviews_df

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
40,4.0,True,"02 4, 2012",ATI21V0A8K25G,0700026657,Hags...,Good game. Good graphics. A bit of a learning ...,Nice..,1328313600,10,NaN,NaN
42,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
44,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
45,5.0,True,"05 27, 2015",A3I3QZXDI4305I,0700026657,Ronald Szarkowski,great game very fun!!!!!,Five Stars,1432684800,NaN,NaN,NaN
46,5.0,True,"05 25, 2015",AED58LHQJHK6E,0700026657,bluevision10s,Great Game. Although at first I could not get...,Great Game. Although at first I could not get ...,1432512000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2423315,5.0,True,"05 1, 2017",AC80XT00V5JIH,B00BSECSP2,TrueBlue1982,good game and mature and devil vs. angle and...,so amazing games star 10,1493596800,NaN,NaN,NaN
2423317,5.0,False,"03 4, 2017",A3B7S8C1W1U2KT,B00BSECSP2,Brlane,after all the backlash i avoided this game for...,the son of sparda returns!,1488585600,NaN,NaN,NaN
2423318,5.0,True,"10 26, 2016",ADWZH3W4TZQJU,B00BSECSP2,Eduardo Mariano,Great!,Five Stars,1477440000,NaN,NaN,NaN
2423617,5.0,True,"01 8, 2016",ARKLONYP75BR3,B00BTNT4G8,Kori,"Simply put, Dynasty Warriors Gundam should've ...",A worthy import for any Gundam or mecha fan.,1452211200,NaN,NaN,NaN


In [32]:
filtered_reviews_df.to_csv(f"{path}/data/filtered_reviews_data.csv", index=False)
filtered_products_df.to_csv(f"{path}/data/filtered_products_data.csv", index=False)